<a href="https://colab.research.google.com/github/QuanGuo/HT-PINN/blob/main/HT_PINN_inverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# import os
# os.chdir('/content/gdrive/My Drive/Colab Notebooks/')
# !pwd

# import torch
# import torch.nn as nn
# from torch.autograd import Variable, grad
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# from utils import *

# import time
# import ast
# import copy

In [ ]:
################ colorbar type for plot ##############
cmap5 = [ 'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
'gist_ncar']


In [ ]:
class PhysicsInformedNN(nn.Module):

  def __init__(self, layers_u, layers_K, input_K=None, inv_params=None, num_pumps=25):
    super(PhysicsInformedNN, self).__init__()
  
    self.layers = layers
  
    self.weights_u, self.biases_u = [], []
    self.weights_K, self.biases_K = self.initialize_NN(layers_K)   
    self.weights = []
    self.biases = []

    self.preds = None

    self.loss = 0.0

    self.loss_list = []   
    # self.loss_dict = {'neum':[], 'diri':[],'u':[],'f':[],'K':[],'around':[],'pump':[]}
    self.loss_container = []
    for i in range(num_pumps): 
      loss_dict = {'neum':[0.0], 'diri':[0.0],'u':[0.0],'f':[0.0],'K':[0.0],'around':[0.0],'pump':[0.0]}
      self.loss_container.append(loss_dict)
      w, b = self.initialize_NN(layers_u)
      self.weights_u.append(w)
      self.biases_u.append(b)
    
      self.weights += w
      self.biases += b
    self.weights += self.weights_K
    self.biases += self.biases_K

  def create_dK(self, K, dx, dy):
    dK_cen = (K[:,2:] - K[:,0:-2])/dx/2
    dK_left = (K[:,1:2] - K[:,0:1])/dx
    dK_right = (K[:,-1:] - K[:,-2:-1])/dx
    dKdx = np.hstack((dK_left, dK_cen, dK_right))

    dK_mid = (K[2:] - K[0:-2,])/dy/2
    dK_top = (K[1:2] - K[0:1])/dy
    dK_bot = (K[-1:] - K[-2:-1])/dy
    dKdy = np.vstack((dK_top, dK_mid, dK_bot))
    
    dKdx = torch.tensor(dKdx, requires_grad=True)
    dKdy = torch.tensor(dKdy, requires_grad=True)
    return dKdx, dKdy

  def initialize_NN(self, layers):        
    weights = []
    biases = []
    num_layers = len(layers) 
    for l in range(0,num_layers-1):
      W = self.xavier_init(size=[layers[l], layers[l+1]])
      b = Variable(torch.zeros([1,layers[l+1]], dtype=torch.float32), requires_grad=True)
      weights.append(W)
      biases.append(b)        
    return weights, biases
      
  def xavier_init(self, size):
    return Variable(nn.init.xavier_normal_(torch.empty(size[0], size[1])), requires_grad=True)
    
  def load_NN(self, layers, params):
    weights = []
    biases = []
    params = torch.tensor(params,dtype=torch.float32)
    num_layers = len(layers)
    i = 1
    for l in range(0,num_layers-2):
      W = torch.zeros([layers[l], layers[l+1]], dtype=torch.float32)
      b = torch.zeros([1,layers[l+1]], dtype=torch.float32)
      W = W + params[i:i+layers[l]]
      b = b + params[i+layers[l]]
      i = i + layers[l] + 1
      weights.append(Variable(W, requires_grad=True))
      biases.append(Variable(b, requires_grad=True))

    output_layer_w = Variable(params[i][:,None], requires_grad=True)
    
    output_layer_b = torch.zeros((1,1), dtype=torch.float32)
    output_layer_b = output_layer_b + params[i+1,0]

    weights.append(output_layer_w)
    biases.append(Variable(output_layer_b,requires_grad=True))

    return weights, biases

  def neural_net(self, x, y, weights, biases):

    num_layers = len(weights) + 1
    H = torch.cat((x,y),1)
    for l in range(0,num_layers-2):
      W = weights[l]
      b = biases[l]
      H = torch.tanh(torch.add(torch.matmul(H, W), b))

    W = weights[-1]
    b = biases[-1]
    Y = torch.add(torch.matmul(H, W), b) #.requires_grad_()

    return Y

  def neural_net_sigmoid(self, x, y, weights, biases):

    num_layers = len(weights) + 1
    H = torch.cat((x,y),1)
    for l in range(0,num_layers-2):
      W = weights[l]
      b = biases[l]
      H = torch.sigmoid(torch.add(torch.matmul(H, W), b))

    W = weights[-1]
    b = biases[-1]
    Y = torch.add(torch.matmul(H, W), b) #.requires_grad_()

    return Y

  def neural_net_relu(self, x, y, weights, biases):
    p = torch.tensor(0.001)
    num_layers = len(weights) + 1
    H = torch.cat((x,y),1)

    W = weights[0]
    b = biases[0]
    H = nn.functional.relu(torch.add(torch.matmul(H, W), b))

    for l in range(1,num_layers-2):
      W = weights[l]
      b = biases[l]
      H = torch.tanh(torch.add(torch.matmul(H, W), b))
    W = weights[-1]
    b = biases[-1]
    # Y = nn.functional.relu(torch.add(torch.matmul(H, W), b))
    # Y = nn.functional.prelu(torch.add(torch.matmul(H, W), b),p)+1
    Y = torch.add(torch.matmul(H, W), b)

    return Y

  def net_u(self, x, y, weights, biases): # head u, including Dirichlet BCs
    u = self.neural_net(x, y, weights, biases)
    return u
  
  def net_K(self, x, y): # hydraulic conductivity K
    K = self.neural_net(x, y, self.weights_K, self.biases_K)
    return K
  
  def get_loc_idx(self, x, y): # hydraulic conductivity K
    x = x.detach().numpy()
    y = y.detach().numpy()

    x_arr = np.linspace(-1,1,self.given_K.shape[0])
    y_arr = np.linspace(-1,1,self.given_K.shape[1])

    xsorted = np.argsort(x_arr)
    xpos = np.searchsorted(x_arr[xsorted], x)
    r = xsorted[xpos]

    ysorted = np.argsort(y_arr)
    ypos = np.searchsorted(y_arr[ysorted], y)
    c = xsorted[ypos]
    # K = given_K[r, c]

    return (r, c)


  def net_du(self, x, y, weights, biases): # first-order derivative match, inlcuding Neumann BCs

    u = self.net_u(x, y, weights, biases)#, self.weights_u, self.biases_u)

    u_x = grad(u.sum(), x, create_graph=True, retain_graph=True)[0]
    u_y = grad(u.sum(), y, create_graph=True)[0]

    return u_x.requires_grad_(True), u_y.requires_grad_(True)

  def net_dK(self, x, y): # first-order derivative of K
    K = self.net_K(x, y)#, self.weights_u, self.biases_u)

    K_x = grad(K.sum(), x, create_graph=True)[0]
    K_y = grad(K.sum(), y, create_graph=True)[0]

    return K_x.requires_grad_(True), K_y.requires_grad_(True)


  def net_f(self, x, y, weights, biases): # general PDE match, usually formulated in higher-order

    u_x, u_y = self.net_du(x, y, weights, biases)
    u_yy = grad(u_y.sum(), y, create_graph=True)[0]
    u_xx = grad(u_x.sum(), x, create_graph=True)[0]

    K = self.net_K(x, y)
    K_x, K_y = self.net_dK(x, y)
    # K_x = grad(K.sum(), x, create_graph=True)[0]
    # K_y = grad(K.sum(), y, create_graph=True)[0]

    f = K*(u_yy + u_xx) + K_x*u_x + K_y*u_y

    return f.requires_grad_(True)

  def forward(self, x_tensors, y_tensors, weights, biases, keys=None):

    if keys is None:
      keys = x_tensors.keys()
    else:
      preds = dict()
      for i in keys:
          preds[i] = None

    for i in keys:

      if i == 'neum':
        dudx_pred, dudy_pred = self.net_du(x_tensors[i], y_tensors[i], weights, biases)
        # flux_pred = self.net_f(x_tensors[i], y_tensors[i])
        preds[i] = dudy_pred

      elif i == 'f':
        f_pred = self.net_f(x_tensors[i], y_tensors[i], weights, biases)
        preds[i] = f_pred

      elif i == 'u':
        u_pred = self.net_u(x_tensors[i], y_tensors[i], weights, biases) 
        preds[i] = u_pred
          
      elif i == 'K':
        # K_pred = nn.functional.relu(self.net_K(x_tensors[i], y_tensors[i]))+1e-4
        K_pred = self.net_K(x_tensors[i], y_tensors[i])

        preds[i] = K_pred
          
      elif i == 'diri':
        diri_pred = self.net_u(x_tensors[i], y_tensors[i], weights, biases) 
        preds[i] = diri_pred

      elif i == 'pump':
        p_pred = self.net_f(x_tensors[i], y_tensors[i], weights, biases)
        preds[i] = p_pred

    return preds

  def loss_func(self, pred_dict, true_dict, pump_id, weights=None):
  
    loss = torch.tensor(0.0, dtype=torch.float32)
    keys = pred_dict.keys()

    if weights is None:
      weights = dict()
      for i in keys:
        weights[i] = 1.0

    for i in keys:
      res = pred_dict[i] - true_dict[i]
      loss += weights[i]*torch.mean(res.pow(2))
      r = torch.mean(res.pow(2)).item()
      self.loss_container[pump_id][i].append(r*weights[i])
    return loss.requires_grad_()
  
  def customized_backward(self, loss, params):
    grads = grad(loss, params, retain_graph=True)
    for vid in range(len(params)):
      params[vid].grad = grads[vid]
    return grads

  def unzip_train_dict(self, train_dict, keys=None):
    if keys is None:
      keys = train_dict.keys()

    x_tensors = dict()
    y_tensors = dict()
    true_dict = dict()

    for i in keys:
      x_tensors[i] = train_dict[i][0]
      y_tensors[i] = train_dict[i][1]
      true_dict[i] = train_dict[i][2]

    return (x_tensors, y_tensors, true_dict)

  def train(self, epoch, data_batch, loss_func, optimizer, pred_keys=None, loss_weights=None, pump_id_list=[0], print_interval=1000):
      
    if pred_keys is None:
      pred_keys= data_batch[0].keys()
    start_time = time.time()
    for i in range(epoch):
      optimizer.zero_grad()
      loss = 0.0
      for pump_id in pump_id_list:
        train_dict = data_batch[pump_id]

        (x_tensors, y_tensors, true_dict) = self.unzip_train_dict(train_dict,pred_keys)
        pred_dict = self.forward(x_tensors, y_tensors, self.weights_u[pump_id], self.biases_u[pump_id], keys=pred_keys)
        loss += loss_func(pred_dict, true_dict, pump_id, loss_weights)

      loss.backward()
      self.callback(loss.detach().numpy().squeeze())

      if np.remainder(len(self.loss_list),print_interval) == 1:
        elapsed = time.time() - start_time
        print('Iter # %d, Loss: %.8f, Time: %.4f' % (len(self.loss_list), self.loss_list[-1], elapsed))
        print_loss = dict()
        for pid in pump_id_list:
          print_loss = "Pump "+ str(pid) + ": "
          for k in ['u','f','K','neum','pump','diri']:
            s = k+":"+str(self.loss_container[pid][k][-1])+"; "
            print_loss += s
          print(print_loss)
        start_time = time.time()  
      
      # g = self.customized_backward(loss, self.weights+self.biases)
      optimizer.step()

    # self.pred_dict = self.forward(x_tensors, y_tensors, keys=pred_keys)
    # self.loss = loss_func(self.pred_dict, true_dict) #.requires_grad_()

  def callback(self, loss):
    self.loss_list.append(loss)

  def coor_shift(self, X, lbs, ubs):
    return 2.0*(X - lbs) / (ubs - lbs) - 1

  def data_loader(self, X, u, lbs, ubs):
              
    X = self.coor_shift(X, lbs, ubs)

    x_tensor = torch.tensor(X[:,0:1], requires_grad=True, dtype=torch.float32)
    y_tensor = torch.tensor(X[:,1:2], requires_grad=True, dtype=torch.float32)

    u_tensor = torch.tensor(u, dtype=torch.float32)
    
    return (x_tensor, y_tensor, u_tensor)

  def predict(self, X_input, pid=0, target='u'):
    x_tensor = torch.tensor(X_input[:,0:1], dtype=torch.float32, requires_grad=True)
    y_tensor = torch.tensor(X_input[:,1:2], dtype=torch.float32, requires_grad=True)
    w = self.weights_u[pid]
    b = self.biases_u[pid]
    pred = None
    if target == 'u':
      pred = self.net_u(x_tensor, y_tensor, w, b).detach().numpy().squeeze()
    elif target == 'du':
      dudx, dudy = self.net_du(x_tensor, y_tensor, w, b)
      return dudx.detach().numpy().squeeze(), dudy.detach().numpy().squeeze()
    elif target == 'f':
      pred = self.net_f(x_tensor, y_tensor, w, b).detach().numpy().squeeze()

    elif target == 'K':
      pred = self.net_K(x_tensor, y_tensor).detach().numpy().squeeze()

    return pred


In [ ]:
# create PINN instance with certain number of LAYERS and HIDDEN UNITS
##################### read lnK realizations (~ 10 fields) ##################
save_label = '0720'   # generation date
field_id = 1   # usually 12 fields: id -> [0-11]

#id=0 for date 0722 with max depth=0.16
#id=1 for date 0720 with max depth=0.16
#id=1 for date 0724 with max depth=3, 128x128
save_path = './data/'+save_label+'/field_'+str(field_id)
isDir = os.path.isdir(save_path)

if isDir:
  print('/field_'+str(field_id)+' existing')
  # file = open(save_path+"hyper_parameters.txt", "r")
  # hyper_param = ast.literal_eval(file.read())
  # file.close()
  # print(hyper_param)
else:
  print('/field_'+str(field_id)+' does not exist') 

In [ ]:
######## first line contains geostatistical and geometry parameters #########
field_file_name = './data/' + save_label +'/fields_'+save_label+'.csv'
df = pd.read_csv(field_file_name, sep=',', header=None)
mu = df.values[0][0]
sigma2 = df.values[0][1]
lx = df.values[0][2]
ly = df.values[0][3]
k = int(df.values[0][4])
nx = int(df.values[0][5])
ny = int(df.values[0][6])

if int(df.values[0][7]) == 1:
  Ctype = "Exponential"
else:
  Ctype = "Gaussian"

print("Covariance func: " + Ctype)
print("mu(lnK):", mu, " m/s")
print("variance:", sigma2)
print("lx:", lx, "; ly:", ly)
print("Resolution: " + str(nx) + "x" + str(ny))

if df.values[0].shape[0] > 11:
  Q = float(df.values[0][8])
  dt_real = float(df.values[0][9])
  dx_real = float(df.values[0][10])
  dy_real = float(df.values[0][11])
  print("Q:", str(Q), " m3/s")
  print("cell size (dx x dy): " + str(dx_real) + "m x " + str(dy_real) + "m")
  print("Time unit:", str(dt_real), " s")
elif df.values[0].shape[0] > 8:
  Q = float(df.values[0][8])
  dx_real = float(df.values[0][9])
  dy_real = float(df.values[0][10])
  print("Q:", str(Q), " m3/s")
  print("cell size (dx x dy): " + str(dx_real) + "m x " + str(dy_real) + "m")

######## select lnK field and reshape ############
logK = df.values[1:,field_id]
logK = logK.reshape((nx,ny)).T
K = np.exp(logK)

#### pump test results on the field ##########
head_file_name = './data/'+save_label+'/heads_'+save_label+'_'+str(field_id+1)+'.csv'
df2=pd.read_csv(head_file_name, sep=',',header=None)
heads = df2.values
num_wells = heads.shape[1]

# solution from FEM is on node and has resolution (nx+1) x (ny+1)
# interpolate center value on each cell as average on 4 corner nodes
heads_at_cell = np.zeros((nx*ny,num_wells))
for ii in range(num_wells):
  head = heads[:,ii]
  head = head.reshape((nx+1,ny+1))
  head = head[1:,:]+head[0:-1,:]
  head = head[:,1:]+head[:,0:-1]
  head = head/4.0
  heads_at_cell[:,ii]=head.T.flatten()

#### read variables ufor inverse ##########
cov_file_name = './data/' + save_label +'/sign_'+save_label+'.csv'
df3=pd.read_csv(cov_file_name, sep=',',header=None)
signs = df3.values[:,0].T   # sign of first variables of each eigenvector
alpha = df3.values[:,field_id+1][:,None] # r.v. used in PCA decomposition

np.savetxt("logK_field.txt",logK)
np.savetxt("alpha_vector.txt",alpha)

In [ ]:
####### plot certain head tomography on 3D surf

# for pid in range(num_wells):
#   idx = np.argmin(heads[:,pid])
#   heads[idx,pid] = 0.5*(heads[idx-1,pid]+heads[idx+1,pid])

si = 2

x = np.linspace(0,1,nx+1)
y = np.linspace(0,1,ny+1)
X,Y = np.meshgrid(x,y)

head = heads[:,si].reshape((nx+1,ny+1))

fig = plt.figure(figsize=(12,5))

# set up the axes for the first plot
ax = fig.add_subplot(1, 2, 1, projection='3d')
surf = ax.plot_surface(X, Y, head, rstride=1, cstride=1, cmap=cm.viridis,
                          linewidth=0, antialiased=False)

ax.set_title('FEM')
ax.view_init(30,45)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

x = np.linspace(0,1,nx)
y = np.linspace(0,1,ny)
X,Y = np.meshgrid(x,y)
Exact = heads_at_cell[:,si].reshape((nx,ny)).T

ax2 = fig.add_subplot(1, 2, 2, projection='3d')
surf = ax2.plot_surface(X, Y, Exact, rstride=1, cstride=1, cmap=cm.viridis,
                          linewidth=0, antialiased=False)
ax2.set_title('Exact')
ax2.set_xlabel('$x$')
ax2.set_ylabel('$y$')
ax2.set_zlabel('$z$')

############### define domain with (0,0) at center ######
Lox, Loy = 1, 1
dx, dy = Lox/nx, Loy/ny

x = np.arange((-Lox/2+dx/2),(Lox/2),dx)
y = np.arange((-Lox/2+dx/2),(Lox/2),dy)

X, Y = np.meshgrid(x,y)

X_star = np.hstack((X.flatten()[:,None], Y.flatten()[:,None]))

################### f PDE (2nd-order derivatives) plot: 2D & 3D ##################
dx,dy=1/nx,1/ny
du_true_cen = (Exact[:,2:] - Exact[:,0:-2])/dx/2
du_true_left = (Exact[:,1:2] - Exact[:,0:1])/dx
du_true_right = (Exact[:,-1:] - Exact[:,-2:-1])/dx
du_true_dx = np.hstack((du_true_left, du_true_cen, du_true_right))

du_true_mid = (Exact[2:] - Exact[0:-2,])/dy/2
du_true_top = (Exact[1:2] - Exact[0:1])/dy
du_true_bot = (Exact[-1:] - Exact[-2:-1])/dy
du_true_dy = np.vstack((du_true_top, du_true_mid, du_true_bot))

du2_cen = ((Exact[:,2:] - Exact[:,1:-1]) - (Exact[:,1:-1] - Exact[:,0:-2]))/dx**2
du2_left = du2_cen[:,0][:,None]
du2_right = du2_cen[:,-1][:,None]
du2dx = np.hstack((du2_left, du2_cen, du2_right))

du2_mid = ((Exact[2:] - Exact[1:-1]) - (Exact[1:-1] - Exact[0:-2]))/dy**2
du2_top = du2_mid[0]
du2_bot = du2_mid[-1]
du2dy = np.vstack((du2_top, du2_mid, du2_bot))

dK_true_cen = (K[:,2:] - K[:,0:-2])/dx/2
dK_true_left = (K[:,1:2] - K[:,0:1])/dx
dK_true_right = (K[:,-1:] - K[:,-2:-1])/dx
dK_true_dx = np.hstack((dK_true_left, dK_true_cen, dK_true_right))

dK_true_mid = (K[2:] - K[0:-2,])/dy/2
dK_true_top = (K[1:2] - K[0:1])/dy
dK_true_bot = (K[-1:] - K[-2:-1])/dy
dK_true_dy = np.vstack((dK_true_top, dK_true_mid, dK_true_bot))

f_res = np.multiply(dK_true_dx,du_true_dx) + np.multiply(dK_true_dy, du_true_dy) + np.multiply(du2dy+du2dx,K)

fig2 = plt.figure(figsize=(12,5))

ax4 = fig2.add_subplot(1, 2, 1)
ax4.pcolor(X,Y,f_res)

ax3 = fig2.add_subplot(1, 2, 2, projection='3d')
surf = ax3.plot_surface(X, Y, f_res, rstride=1, cstride=1, cmap=cm.viridis,
                          linewidth=0, antialiased=False)
ax3.set_xlabel('$x$')
ax3.set_ylabel('$y$')
ax3.set_zlabel('$f$')

print(np.max(f_res))
print(np.min(f_res))
print(3600/1.25/1.25*0.001)


In [ ]:
#################  K measurement cell id ##################
# dtb: dist (in num of cells) to boundary, e.g., 3, 4, 5..
# dtm: dist (in ratio) between nearest measurement, e.g., 1/8, 1/4 ..
##  resolution    dtb     dtm       shift
##   32x32         3      1/6         0 
##   32x32         3      1/7         1 
##   51x51         3      1/6         4
##   64x64         3      1/6         4 
##   64x64         3      1/7         2

# dtb = 1

# dtm = 1/6.0
# if nx == 32:
#   shift = 1
# else:
#   shift = 0

# xloc = np.arange(dtb, nx-dtb, int((nx-2*dtb)*dtm)) + shift
# yloc = np.arange(dtb, nx-dtb, int((ny-2*dtb)*dtm)) + shift

# xloc = np.repeat(xloc,len(xloc))
# yloc = np.tile(yloc,len(yloc))

# # xloc = np.floor(np.linspace(1, nx-2, 7)).astype(int)
# # yloc = np.floor(np.linspace(1, ny-2, 7)).astype(int)

# # xloc, yloc = np.repeat(xloc,len(yloc)), np.tile(yloc,len(xloc))

# K_measure_id = xloc + yloc*ny

dtb = 1

dtm = 1/5.0
if nx == 32:
  shift = 1
else:
  shift = 0
  # shift = 1

xloc = np.arange(dtb, nx-dtb, int((nx-2*dtb)*dtm)) + shift
yloc = np.arange(dtb, nx-dtb, int((ny-2*dtb)*dtm)) + shift
# print(xloc)
xloc, yloc = np.repeat(xloc,len(yloc)), np.tile(yloc,len(xloc))
K_measure_id = xloc + yloc*ny

# 128x128: dtb = 14~15
# 256x256: dtb = 28~29
dtb=14
xloc2 = np.floor(np.linspace(dtb, nx-dtb-1, 5)).astype(int)
yloc2 = np.floor(np.linspace(dtb, ny-dtb-1, 5)).astype(int)
print(xloc2)

xloc2, yloc2 = np.repeat(xloc2,len(yloc2)), np.tile(yloc2,len(xloc2))
K_extra_id = xloc2 + yloc2*ny


K_measure_id = np.hstack((K_measure_id,K_extra_id))
print(len(K_measure_id))


# N_K = 40
# K_measure_id = np.random.choice(nx*ny, N_K, replace=False)

# Km_id = np.loadtxt(save_path+"/npump_9/K_measure_id_49.txt").astype(int)
# print(Km_id)
# K_measure_id = Km_id
################ K measurments well ###################

fig2,ax2 = plt.subplots(figsize=(7,6))

#cmap5: 13 "jet"
im2 = ax2.pcolor(X,Y,logK, cmap='jet')
# plt.colorbar()
ax2.set_xlabel('x')
ax2.set_ylabel('y')
fig.colorbar(im2, ax=ax2, orientation='vertical')
ax2.scatter(X_star[K_measure_id,0], X_star[K_measure_id,1], marker='v', zorder=1, alpha= 1, c='m', s=30)
ax2.set_title("lnK field")
print(K_measure_id)
# ############### load K_measure_id ################


In [ ]:
#################  well network cell id ##################
well_id = np.arange(num_wells)

# pump well matrix-wise index (index on x and y axis)
xloc = np.arange(int(nx/4),int(nx*3/4+1),int(nx/8))
yloc = np.arange(int(ny/4),int(ny*3/4+1),int(ny/8))

xloc = np.repeat(xloc,5)
yloc = np.tile(yloc,5)

# pump well index
pump_cell_idx = xloc + yloc*ny

#################  boundary cell id ##################
id1 = np.where(X.flatten() == min(x))
id2 = np.where(X.flatten() == max(x))

id3 = np.where(Y.flatten() == min(y))
id4 = np.where(Y.flatten() == max(y))
xbound_idx = np.unique(np.hstack((id1,id2)))
ybound_idx = np.unique(np.hstack((id3,id4)))

#################  relax region cell id ##################
# points around pump (relax region for PDE or extra monitor cells)
r = 3
pump_row_idx = np.repeat(pump_cell_idx[:,None],2*r-1,1)
pump_row_idx = pump_row_idx - np.arange(-r+1,r)
around_idx = [pump_row_idx]
for i in range(1,r):
  around_idx = [pump_row_idx-nx*i] + around_idx + [nx*i+pump_row_idx]
around_idx = np.hstack(around_idx)
around_idx = np.delete(around_idx,int(((2*r-1)**2-1)/2),1)

################ individual pump well ###################
ii = 9

head = heads_at_cell[:,ii].reshape((nx,ny))

fig, ax = plt.subplots(1, 1, figsize=(6,6))

im = ax.pcolor(X,Y,head)
# ax.add_colorbar
ax.set_xlabel('x')
ax.set_ylabel('y')

ax.scatter(X_star[pump_cell_idx[ii],0], X_star[pump_cell_idx[ii],1], zorder=1, alpha= 0.5, c='r', s=10)
ax.scatter(X_star[np.delete(pump_cell_idx,ii),0], X_star[np.delete(pump_cell_idx,ii),1], zorder=1, alpha= 0.5, c='b', s=10)
ax.scatter(X_star[around_idx[ii],0], X_star[around_idx[ii],1], zorder=1, alpha= 0.7, c='m', s=10)
ax.scatter(X_star[xbound_idx,0], X_star[xbound_idx,1], marker='v', zorder=1, alpha= 1, c='r', s=10)
ax.scatter(X_star[ybound_idx,0], X_star[ybound_idx,1], marker='v', zorder=1, alpha= 1, c='k', s=10)
ax.set_title('Head Tomography')

In [ ]:
############# data used for all pump tests ##########
############# K measurements ##########
############# Diri & Neum BCs ##########

K_star = K.flatten()[:,None]
K_train = K_star[K_measure_id]
X_K_train = X_star[K_measure_id]

# Domain bounds   
lbs = np.array([min(x),min(y)])
ubs = np.array([max(x),max(y)])
    
################### Dirichlet BCs (left & right bound)  ##########
N_diri = 64 # No. of point for Dirichlet BCs
# left (x=0) and right (x=1)
xx = X_star[xbound_idx]
uu = np.zeros((xx.shape[0],1))

X_diri_train = xx
diri_train = uu

# X_diri_train, diri_train = random_choice_sample([X_diri_train, diri_train], N_diri)
# X_diri_train, diri_train = random_choice_sample([xx2, uu2], N_diri)

################### Neumann BCs (top & bottom bound)  #############
N_neum = 64  # No. of points for Neumann BCs
# bottom (y = 0) and top (y=1)
yy = X_star[ybound_idx]
du = np.zeros((yy.shape[0],1))

X_neum_train = yy
neum_train = du

# X_neum_train, neum_train = random_choice_sample([X_neum_train, neum_train], N_neum)



In [ ]:
hnu = 20 # hidden unit number of net u
hnK = 20 # hiddent unit number of net K
layers = [2, hnu, hnu, hnu, hnu, hnu, hnu, 1]
layers_K = [2, hnK, hnK, hnK, hnK, hnK, hnK, 1]

model =PhysicsInformedNN(layers,layers_K)

In [ ]:
# xlin = np.linspace(0,nx-1,64,dtype=np.int)
xlin = np.linspace(0,nx,65,dtype=np.int)
xlin[0] = xlin[0]+1
xlin[-1] = xlin[-1]-1
ylin = np.linspace(0,ny,65,dtype=np.int)
ylin[0] = ylin[0]+1
ylin[-1] = ylin[-1]-1
print(xlin)

xlin = np.repeat(xlin,xlin.shape[0])
ylin = np.tile(ylin,ylin.shape[0])

X_id_65x65  = xlin + ylin*ny
# X_f_space = X_star[X_id_65x65]
# print(X_f_id.shape)

In [ ]:
neum_data = model.data_loader(X_neum_train, neum_train, lbs, ubs)
diri_data = model.data_loader(X_diri_train, diri_train, lbs, ubs)
K_data = model.data_loader(X_K_train, K_train, lbs, ubs)
Qp = 40
data_batch = []

for jj in range(num_wells):
  max_head_at_cell = np.max(np.abs(heads_at_cell[:,jj]))
  # print(max_head_at_cell)
  heads_at_cell_scaled = heads_at_cell[:,jj][:,None]/max_head_at_cell*0.01
  u_star = heads_at_cell_scaled
  # print(np.min(u_star))
  # pump well and pump rate 
  X_pump_train = X_star[pump_cell_idx[jj]][None,:]
  pump_train = np.array([[Qp]])

  # monitor wells and head, besides known BCs
  X_u_train = X_star[np.delete(pump_cell_idx,jj)]
  u_train = u_star[np.delete(pump_cell_idx,jj)]

  # cells outside pump (relax) region set for PDE constraints
  region_idx = np.hstack((pump_cell_idx[jj],around_idx[jj],xbound_idx,ybound_idx))

  # X_f_id = np.setdiff1d(X_id_65x65,region_idx)
  # X_f_space = X_star[X_f_id]
  X_f_space = np.delete(X_star, region_idx, 0)
  # f_id = np.random.choice(X_f_space.shape[0], 900, replace=False)
  X_f_train = X_f_space[::1]
  f_train = np.zeros((X_f_train.shape[0],1))

  ################ random measurements outside relax region #########
  # N_K = 40
  # K_measure_space_id = np.delete(space_id, region_idx, 0)
  # measure_id = np.random.choice(nx*ny-region_idx.shape[0], N_K, replace=False) 
  # K_measure_id = K_measure_space_id[measure_id]

  
  # X_around_train = X_star[around_idx[jj]]
  # around_train = u_star[around_idx[jj]]
  # around_train = np.zeros((X_around_train.shape[0],1))
  # around_data = model.data_loader(X_around_train, around_train, lbs, ubs)
  ##############  data around pump wells  ##############
  
  f_data = model.data_loader(X_f_train, f_train, lbs, ubs)
  u_data = model.data_loader(X_u_train, u_train, lbs, ubs)
  pump_data = model.data_loader(X_pump_train, pump_train, lbs, ubs)

  # Assemble data batch containing training data
  # training data is in dict format: key: name -> value: (x,y,val)
  train_dict = {
    'neum': neum_data,
    'diri': diri_data,
    'u': u_data,
    'f': f_data,
    'K': K_data,
    'pump': pump_data
  }

  data_batch.append(train_dict)
  
fig,ax = plt.subplots(figsize=(7,6))

plt.pcolor(X,Y,np.reshape(heads_at_cell[:,-1],[nx,ny]))
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y')

ax.scatter(X_f_train[:,0], X_f_train[:,1], marker='v', zorder=1, alpha= 1, c='k', s=10)
print(X_u_train.shape)
print(X_f_train.shape)

In [ ]:
# data_batch[pump_id_list[-1]]['u'][2]

In [ ]:
# pump_id_list = [i for i in range(num_wells)]

# X: 12, 0, 4, 20, 24
# cross: 12, 2, 10, 14, 22
# inner 3x3: 6, 7, 8, 11, 12, 13, 14, 15, 16 
# outer 3x3: 12, 2, 10, 14, 22, 0, 4, 20, 24

# pump_id_list = [12, 2, 10, 14, 22, 0, 4, 20, 24]
# pump_id_list = [0, 2, 4, 10, 12, 14, 20, 22, 24]

pump_id_list = [0, 4, 12, 20, 24]

# pump_id_list = [24]
npump = len(pump_id_list)
# npump=5

In [ ]:
########## load existing model ##############

NK = "_"+str(len(K_measure_id))

for i in pump_id_list:
  params_u = np.loadtxt(save_path+"/npump_"+str(npump)+"/model_u"+NK+"_npump_"+str(npump)+"_"+str(i)+".txt")
  model.weights_u[i], model.biases_u[i] = model.load_NN(layers, params_u)

params_K = np.loadtxt(save_path+"/npump_"+str(npump)+"/model_K"+NK+"_npump_"+str(npump)+"_.txt")
model.weights_K, model.biases_K= model.load_NN(layers_K, params_K)

# NK = ""
# for i in pump_id_list:
#   params_u = np.loadtxt(save_path+"/model_u"+NK+"_"+str(i)+".txt")
#   model.weights_u[i], model.biases_u[i] = model.load_NN(layers, params_u)

# params_K = np.loadtxt(save_path+"/model_K"+NK+".txt")
# model.weights_K, model.biases_K= model.load_NN(layers_K, params_K)

In [ ]:
# training set up: hyper-parameters

# # weights of each kind of loss
# loss_weights = {
#   'f': 50.0*200,
#   'u': 20000.0*200,
#   'neum': 10000.0*200,
#   'pump': 1.0,
#   'K': 100.0*200,
#   'diri': 20000.0*200
# }

# weights of each kind of loss
loss_weights = {
  'f': 50.0,
  'u': 10000.0,
  'neum': 10000.0,
  'pump': 1.0,
  'K': 100.0,
  'diri': 20000.0
}

# loss type aggregated in total loss
pred_ks = ['diri', 'neum', 'u', 'pump','f','K']
# pred_ks = ['u','diri','neum','K','pump']


# networks weights to tune
biases_train = []
weights_train = []

for pid in pump_id_list:
  weights_train += model.weights_u[pid]
  biases_train += model.biases_u[pid]

biases_train += model.biases_K
weights_train += model.weights_K

# loss change print intervals
p_intervals = 300

In [ ]:
# # training process
# optimizer = torch.optim.Adam(params=weights_train+biases_train, lr=1e-3)

# start_time = time.time()

# model.train(901, data_batch, model.loss_func, optimizer, pred_ks, loss_weights, pump_id_list, p_intervals)

# # optimizer.lt=1e-5
# # model.train(6000, data_batch, model.loss_func, optimizer, pred_ks, loss_weights, pump_id_list, p_intervals)
# # optimizer.lt=1e-5
# # model.train(6000, data_batch, model.loss_func, optimizer, pred_ks, loss_weights, pump_id_list, p_intervals)

# elapsed = time.time() - start_time
# print('Training time: %.4f' % (elapsed))


In [ ]:
# # pred_ks = ['diri', 'neum', 'u', 'pump','f','K']
# optimizer.lr=1e-6

# start_time = time.time()
# model.train(10000, data_batch, model.loss_func, optimizer, pred_ks, loss_weights, pump_id_list, 1000)
# elapsed = time.time() - start_time
# print('Training time: %.4f' % (elapsed))

In [ ]:
X_pred= model.coor_shift(X_star, lbs, ubs)
error_u_list = []
for ti in pump_id_list:

  
  u_true = heads_at_cell[:,ti]
  u_pred = model.predict(X_pred, pid=ti, target='u')/0.01*np.max(np.abs(u_true))
  u_err = u_pred-u_true
  error_u = np.linalg.norm(u_err,2)/np.linalg.norm(u_true,2)
  error_u_list.append(error_u)

print(error_u_list)
print("Min Err: ", min(error_u_list))
print("Max Err: ", max(error_u_list))
print("mean Err: ", np.mean(error_u_list))
print("std Err: ", np.std(error_u_list))

In [ ]:
ti = pump_id_list[2]
X_pred= model.coor_shift(X_star, lbs, ubs)
u_true = heads_at_cell[:,ti]
u_pred = model.predict(X_pred, pid=ti, target='u')/0.01*np.max(np.abs(u_true))

u_err = u_pred-u_true
error_u = np.linalg.norm(u_err,2)/np.linalg.norm(u_true,2)
print('Error u: %e' % (error_u))

u_pred = u_pred.reshape((nx,ny))
u_true = u_true.reshape((nx,ny))

################### head plot: 3D ##################
plot_3D(x,y,u_true, 'True')
plot_3D(x,y,u_pred, 'Prediction')


In [ ]:
################### hydraulic conductivity colormap plot: 2D ##################
K_pred = model.predict(X_pred, pid=ti, target='K')
K_true = K.flatten()

K_err = K_true - K_pred
error_K = np.linalg.norm(K_err,2)/np.linalg.norm(K_true,2)
print(error_K)
K_pred = K_pred.reshape((nx,ny))

minlK, maxlK = np.min(logK), np.max(logK)


fig,ax = plt.subplots(figsize=(7,6))
im = ax.pcolor(X,Y,logK,cmap=cmap5[13])
im.set_clim(minlK, maxlK )
fig.colorbar(im, ax=ax)
plt.xlabel('x')
plt.ylabel('y')
ax.scatter(X_star[K_measure_id,0], X_star[K_measure_id,1], zorder=1, alpha= 0.8, c='r', s=15)
ax.set_title('True')
logK_pred = np.nan_to_num(np.log(K_pred),nan=minlK)
# K_pred = np.log(K_pred)
fig,ax = plt.subplots(figsize=(7,6))
im = ax.pcolor(X,Y, logK_pred,cmap=cmap5[13])
im.set_clim(minlK, maxlK )

fig.colorbar(im, ax=ax)
plt.xlabel('x')
plt.ylabel('y')
ax.scatter(X_star[K_measure_id,0], X_star[K_measure_id,1], zorder=1, alpha= 0.8, c='r', s=15)
ax.set_title('Prediction')


In [ ]:
fig,ax = plt.subplots(figsize=(7,6))

# threshold 10%
thres = 0.1
K_len = maxlK-minlK

# acc = np.divide(abs(logK-K_pred),abs(K))
acc = abs(logK-logK_pred)/K_len


print(sum(sum(acc<thres))/(nx*ny))
plt.pcolor(X,Y,acc<thres,cmap=plt.cm.BuPu_r)

plt.colorbar()
plt.xlabel('x')
plt.ylabel('y')
plt.title('Absolute Error')

In [ ]:
################### head countour plot: 2D ##################

scaler_min = np.min(u_true)
scaler_max = np.max(u_true)
scaler_len = scaler_max - scaler_min
# print(scaler_len)

u_scaled = (u_pred - np.min(u_pred))/(np.max(u_pred) - np.min(u_pred))
u_scaled = u_scaled * scaler_len+scaler_min

mask_min = np.min(u_scaled)
# print(mask_min)
# Exact = u_true
# Exact[Exact<mask_min] = mask_min

# plot_3D(x,y,u_scaled, 'Prediction')
# plot_3D(x,y,Exact, 'True')

# compare_true_pred(Exact, u_scaled, x, y)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))
lvls = np.linspace(scaler_min,scaler_max,7)

CT = ax1.contour(X, Y, u_true, levels=lvls)
ax1.clabel(CT,inline=True)
ax1.set_title('True')

CP = ax2.contour(X, Y, u_pred,levels=lvls,linestyles='dashed')
ax2.clabel(CP,inline=True)
ax2.set_title('Prediction')


In [ ]:
################### quiver (1st-order gradient arrow) plot: 2D ##################
# dx, dy = 5.0, 5.0

du_true_cen = (u_true[:,2:] - u_true[:,0:-2])/dx/2
du_true_left = (u_true[:,1:2] - u_true[:,0:1])/dx
du_true_right = (u_true[:,-1:] - u_true[:,-2:-1])/dx
du_true_dx = np.hstack((du_true_left, du_true_cen, du_true_right))

du_true_mid = (u_true[2:] - u_true[0:-2,])/dy/2
du_true_top = (u_true[1:2] - u_true[0:1])/dy
du_true_bot = (u_true[-1:] - u_true[-2:-1])/dy
du_true_dy = np.vstack((du_true_top, du_true_mid, du_true_bot))

dudx, dudy = model.predict(X_pred, pid=ti, target='du')
error_neum = np.linalg.norm(dudy[ybound_idx],2)/dudy[ybound_idx].shape[0]
print('Error neum: %e' % (error_neum))

dudx = dudx.reshape((nx,ny))
dudy = dudy.reshape((nx,ny))
fig2, (ax3, ax4) = plt.subplots(1, 2, figsize=(13,5))

im2 = ax3.pcolor(X,Y,u_true)
cbar = fig2.colorbar(im2, ax=(ax3, ax4), shrink=0.95)

ax3.quiver(X,Y,du_true_dx, du_true_dy)
ax3.set_title('True')

ax4.pcolor(X,Y,u_pred)
ax4.quiver(X,Y,dudx, dudy)
ax4.set_title('Prediction')
# im2 = ax4.pcolor(X,Y,u_true)
# cbar = fig2.colorbar(im2, ax=(ax3, ax4), shrink=0.95)

In [ ]:
################### f PDE (2nd-order derivatives) plot: 2D & 3D ##################
du2_cen = ((u_true[:,2:] - u_true[:,1:-1]) - (u_true[:,1:-1] - u_true[:,0:-2]))/dx**2
du2_left = du2_cen[:,0][:,None]
du2_right = du2_cen[:,-1][:,None]
du2dx = np.hstack((du2_left, du2_cen, du2_right))

du2_mid = ((u_true[2:] - u_true[1:-1]) - (u_true[1:-1] - u_true[0:-2]))/dy**2
du2_top = du2_mid[0]
du2_bot = du2_mid[-1]
du2dy = np.vstack((du2_top, du2_mid, du2_bot))

dK_true_cen = (K[:,2:] - K[:,0:-2])/dx/2
dK_true_left = (K[:,1:2] - K[:,0:1])/dx
dK_true_right = (K[:,-1:] - K[:,-2:-1])/dx
dK_true_dx = np.hstack((dK_true_left, dK_true_cen, dK_true_right))

dK_true_mid = (K[2:] - K[0:-2,])/dy/2
dK_true_top = (K[1:2] - K[0:1])/dy
dK_true_bot = (K[-1:] - K[-2:-1])/dy
dK_true_dy = np.vstack((dK_true_top, dK_true_mid, dK_true_bot))

f_res = np.multiply(dK_true_dx,du_true_dx) + np.multiply(dK_true_dy, du_true_dy) + np.multiply(du2dy+du2dx,K)

f_pred = model.predict(X_pred, pid=ti, target='f')

region_idx = np.unique(np.hstack((pump_cell_idx[ti],around_idx[ti],xbound_idx,ybound_idx)))
f_err = np.delete(f_pred, region_idx, 0)
error_pde = np.linalg.norm(f_err,2)/f_err.shape[0]
print('Error pde: %e' % (error_pde))

f_pred = f_pred.reshape((nx,ny))

fig3, (ax5, ax6) = plt.subplots(1, 2, figsize=(12,5))
ax5.pcolor(X,Y,f_pred)
ax5.set_title('Prediction')
ax6.pcolor(X,Y,f_res)
ax6.set_title('True')

ax5.scatter(X_star[around_idx[ti],0], X_star[around_idx[ti],1], zorder=1, alpha= 0.6, c='y', s=10)
ax6.scatter(X_star[around_idx[ti],0], X_star[around_idx[ti],1], zorder=1, alpha= 0.6, c='y', s=10)


plot_3D(x,y,f_pred,figname='prediction')
plot_3D(x,y,f_res,figname='true')
print(np.max(f_res))
print(np.min(f_res))
print(Q/dx/dy)

In [ ]:
# ############## save well-trained model: u, K, and measured K id
# if not isDir:
#   try:
#     os.mkdir(save_path)
#     os.mkdir(save_path+"/npump_"+str(npump))
#   except:
#     FileExistsError

# NK = "_"+str(len(K_measure_id))
# m = model
# for i in pump_id_list:
#   weights = torch.ones(1,hnu)*i

#   for j in range(len(layers)-2):
#     w = m.weights_u[i][j]
#     b = m.biases_u[i][j]
#     weights = torch.vstack((weights,w,b))

#   weights = torch.vstack((weights,m.weights_u[i][j+1].T,torch.ones(1,hnu)*m.biases_u[i][j+1]))
#   weights=weights.detach().numpy()


#   path_name = save_path+"/npump_"+str(npump)+"/model_u"+NK+"_npump_"+str(len(pump_id_list))+"_"+str(i)+".txt"
#   np.savetxt(path_name,weights)

# weights = torch.ones(1,hnK)*i

# for j in range(len(layers)-2):
#   w = m.weights_K[j]
#   b = m.biases_K[j]
#   weights = torch.vstack((weights,w,b))

# weights = torch.vstack((weights,m.weights_K[j+1].T,torch.ones(1,hnK)*m.biases_K[j+1]))
# weights=weights.detach().numpy()

# path_name = save_path+"/npump_"+str(npump)+"/model_K"+NK+"_npump_"+str(len(pump_id_list))+"_"+".txt"
# np.savetxt(path_name,weights)

# path_name = save_path+"/npump_"+str(npump)+"/K_measure_id"+NK+".txt"
# np.savetxt(path_name,K_measure_id.astype(int))

In [ ]:
# ############## save hyper-parameters for reproduce ##################
# hyper_param = copy.copy(loss_weights)
# hyper_param['Qp'] = Qp
# hyper_param['r'] = r
# hyper_param['hnK'] = hnK
# hyper_param['hnu'] = hnu
# hyper_param['lenK'] = len(layers_K)
# hyper_param['lenu'] = len(layers)
# hyper_param['act_K'] = 'tanh'
# hyper_param['act_u'] = 'tanh'
# hyper_param['out_K'] = 'linear'
# hyper_param['out_u'] = 'linear'

# f = open(save_path+"hyper_parameters.txt","w")
# f.write(str(hyper_param))
# f.close()